<div class="viewer--28ZF"><h1><font color="#ff5533">&gt;</font> Описание данных</h1><hr><h5>Таблица user_data</h5><p class="p--1qUA">Cодержит информацию о всех пользователях соц.сети</p><table class="table table-bordered"><tbody><tr><td>Field name</td><td>Overview</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">age</span></td><td>Возраст пользователя (в профиле)</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">city</span></td><td>Город пользователя (в профиле)</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">country</span></td><td>Страна пользователя (в профиле)</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">exp_group</span></td><td>Экспериментальная группа: некоторая зашифрованная категория</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">gender</span></td><td>Пол пользователя</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">user_id</span></td><td>Уникальный идентификатор пользователя</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">os</span></td><td>Операционная система устройства, с которого происходит пользование соц.сетью</td></tr><tr><td><b>source</b></td><td>Пришел ли пользователь в приложение с органического трафика или с рекламы</td></tr></tbody></table><hr><h5>Таблица post_text_df</h5><p class="p--1qUA">Содержит информацию о постах и уникальный ID каждой единицы с соответствующим ей текстом и топиком</p><table class="table table-bordered"><tbody><tr><td><b>Field name</b></td><td><b>Overview</b></td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">id</span></td><td>Уникальный идентификатор поста</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">text</span></td><td>Текстовое содержание поста</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">topic</span></td><td>Основная тематика</td></tr></tbody></table><hr><h5>Таблица feed_data</h5><p class="p--1qUA">Содержит историю о просмотренных постах для каждого юзера в изучаемый период.</p><p class="p--1qUA"><b>Внимание: Таблица ООООЧЕНЬ большая. Рекомендуется не загружать ее полностью, иначе все упадет.</b></p><table class="table table-bordered"><tbody><tr><td><b>Field name</b></td><td><b>Overview</b></td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">timestamp</span></td><td>Время, когда был произведен просмотр</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">user_id</span></td><td>id пользователя, который совершил просмотр</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">post_id</span></td><td>id просмотренного поста</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">action</span></td><td>Тип действия: просмотр или лайк</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">target</span></td><td>1 у просмотров, если почти сразу после просмотра был совершен лайк, иначе 0. У действий like пропущенное значение.</td></tr></tbody></table><hr><p class="p--1qUA"><div class="copy-button-wrapper--1rIz"></div></div></div>

### Таргетная переменная
Таргетной переменной является колонка `target` из таблицы `feed_action`. Суть разницы между `target` и `action` в том, что `user` может посмотреть `post`, но поставить лайк позже, чем посмотрел (такое поведение учитывает `action`, а `target` указывает на "посмотрел и тут же лайкнул")
***

In [1]:
import os

os.chdir("../")  # Укажите путь к корню проекта
print(os.getcwd())  # Проверьте, что путь изменился

c:\Users\MishaV\Desktop\karpov\ML\L22


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

In [3]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [4]:
# Открываем соединение с использованием stream_results
with engine.connect().execution_options(stream_results=True) as connection:
    # Открываем файл CSV для записи
    with open("./data/raw/feed_data.csv", "w", encoding="utf-8", newline="") as csvfile:
        # Читаем данные из базы данных по частям
        for chunk in pd.read_sql(
            "SELECT * FROM feed_data", con=connection, chunksize=200_000
        ):
            # Записываем данные в CSV файл
            chunk.to_csv(
                csvfile, index=False, header=csvfile.tell() == 0
            )  # Добавляем заголовок только для первой части

***
## Обработка пользователей

In [3]:
# users = pd.read_sql("SELECT * FROM user_data", con=engine, index_col="user_id")
users = pd.read_csv("./data/raw/users.csv", index_col="user_id")
print(users.shape)
users.head()

(163205, 7)


,gender,age,country,city,exp_group,os,source
user_id,,,,,,,
200,1,34,Russia,Degtyarsk,3,Android,ads
201,0,37,Russia,Abakan,0,Android,ads
202,1,17,Russia,Smolensk,4,Android,ads
203,0,18,Russia,Moscow,1,iOS,ads
204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [ ]:
users.nunique()

In [ ]:
hist_cols = users.columns.drop("city")
fig, axes = plt.subplots(
    nrows=len(hist_cols), figsize=(14, 10), gridspec_kw={"hspace": 0.7, "wspace": 0.7}
)

# Проходимся по столбцам и осям
for col, ax in zip(hist_cols, axes):
    sns.histplot(data=users, x=col, ax=ax)

***
## Обработка постов

Новые фичи:
1. Длина текста (количество символов в тексте)
2. Берем первые три предложения каждого текста, считаем количество присутствий каждого слова во всех текстах (из 3-ех предложений), отбираем вручную 10 популярных слов

In [ ]:
# posts = pd.read_sql("SELECT * FROM post_text_df", con=engine, index_col="post_id")
posts = pd.read_csv("./data/raw/posts.csv", index_col="post_id")
print(posts.shape)
posts.head()

In [ ]:
posts.nunique()

In [5]:
posts["length"] = posts["text"].str.len()
posts = posts.drop("text", axis=1)

***
## Обработка активностей



In [ ]:
pd.read_sql("SELECT * FROM feed_data LIMIT 5", con=engine)

### Некоторые расчетные характеристики

- Временной промежуток выборки (самая ранняя активность, самая поздняя активность):
<div style="text-align: center">(2021.10.01 06:01:40; 2021.12.29 23:51:06)</div>

- Количество записей:
<div style="text-align: center">76 892 800</div>

- Среднее количество активностей пользователя в день: 
  - как вариант, можно посчитать для субботы и воскресения отдельно. 
<div style="text-align: center">user_avg_actions_per_day</div>

- Частота активностей пользователя относительно всех записей:
<div style="text-align: center">user_target_ratio</div>

- Частота лайков пользователя относительно всех просмотров пользователя:
<div style="text-align: center">user_like_ratio</div>

In [ ]:
# pd.read_sql("SELECT max(timestamp), min(timestamp) FROM feed_data", con=engine)

In [ ]:
# feed_data_count = pd.read_sql("SELECT count(*) FROM feed_data", con=engine).iloc[0, 0]
# feed_data_count

Среднее количество активностей пользователя в день

In [ ]:
# user_avg_actions_per_day = pd.read_sql("""
#     WITH daily_actions AS (
#         SELECT
#             user_id,
#             DATE(timestamp) AS action_date,
#             COUNT(*) AS actions_per_day
#         FROM
#             feed_data
#         GROUP BY
#             user_id,
#             DATE(timestamp)
#         )
#     SELECT
#         user_id,
#         AVG(actions_per_day) AS avg_actions_per_day
#     FROM
#         daily_actions
#     GROUP BY
#         user_id;
#     """, con=engine, index_col="user_id")
# user_avg_actions_per_day.to_csv(r".\data\user_avg_actions_per_day.csv")
user_avg_actions_per_day = pd.read_csv(
    "../data/extracted/user_avg_actions_per_day.csv", index_col="user_id"
)
user_avg_actions_per_day.head()

In [ ]:
plt.figure(figsize=(14, 10))
sns.boxplot(y=user_avg_actions_per_day["avg_actions_per_day"], orient="y")

<span style="color:red">Можно будет попробовать убрать выбросы</span>

Частота активностей пользователя относительно всех записей

In [ ]:
# user_target_ratio = pd.read_sql("""
#     SELECT
#         user_id,
#         COUNT(*)::float / (SELECT count(*) FROM feed_data) AS target_ratio
#     FROM
#         feed_data
#     GROUP BY
#         user_id;
#     """, con=engine, index_col="user_id")
# user_target_ratio.to_csv(r"..\data\extracted\user_target_ratio.csv")
user_target_ratio = pd.read_csv(
    r"..\data\extracted\user_target_ratio.csv", index_col="user_id"
)
user_target_ratio.head()

Частота лайков пользователя относительно всех просмотров пользователя

In [ ]:
# user_like_ratio = pd.read_sql("""
#     SELECT
#         user_id,
#         COUNT(CASE WHEN target = 1 THEN 1 END)::float / COUNT(*) AS target_ratio
#     FROM
#         feed_data
#     GROUP BY
#         user_id;
#     """, con=engine, index_col="user_id")
# user_like_ratio.to_csv(r"..\data\extracted\user_like_ratio.csv")
user_like_ratio = pd.read_csv(
    r"..\data\extracted\user_like_ratio.csv", index_col="user_id"
)
user_like_ratio.head()

In [ ]:
plt.figure(figsize=(14, 12))
sns.boxplot(y=user_target_ratio["target_ratio"], orient="y")

In [ ]:
users.isna().sum().sum()

In [ ]:
posts.isna().sum().sum()

***
## Отбор данных для обучения и тестирования
В `feed_data` ~76 млн записей. Всех их не использовать в обучении и тестировании. Возьмем только 4 млн.

In [ ]:
# with engine.connect() as connection:
#     connection.execute("SELECT setseed(0.42);")
#     selection = pd.read_sql("""
#         SELECT * FROM feed_data WHERE 4000000.0 / (SELECT count(*) FROM feed_data) > random();
#         """, con=engine)
# selection.to_csv(r"..\data\raw\feed_data_selection.csv")
selection = pd.read_csv(r"./data/raw/selection.csv", index_col="user_id")
print(selection.shape)
selection.head()

Проверим, все ли пользователи вошли в выборку.

In [21]:
data = pd.merge(selection, users, left_on="user_id", right_index=True)
data = pd.merge(data, posts, left_on="post_id", right_index=True)
data = pd.merge(data, user_like_ratio, left_on="user_id", right_index=True)
data = pd.merge(data, user_target_ratio, left_on="user_id", right_index=True)
# data = pd.merge(selection, user_avg_actions_per_day, left_on="user_id", right_index=True)

In [ ]:
data